In [2]:
from elasticsearch import Elasticsearch

In [3]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.indices.create('es0', body={
    "mappings": {
        "properties": {
            "name": {
                "type": "text",
                "analyzer": "ik_max_word",
                "fields": {
                    "raw": {
                        "type": "keyword"
                    }
                }
            },
            "vote": {
                "type": "long",
            }
        }
    }
})

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es0'}

In [4]:
es.index('es0', id=0, body={"name": "Java集合详解",
                            "vote": 100})
es.index('es0', id=1, body={"name": "Java多线程详解,Java锁详解",
                            "vote": 30})

{'_index': 'es0',
 '_type': '_doc',
 '_id': '1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

The newly computed score is combined with the score of the query. The parameter boost_mode defines how:

| `multiply` | query score and function score is multiplied (default)  |
| ---------- | ------------------------------------------------------- |
| `replace`  | only function score is used, the query score is ignored |
| `sum`      | query score and function score are added                |
| `avg`      | average                                                 |
| `max`      | max of query score and function score                   |
| `min`      | min of query score and function score                   |

In [5]:
body = {
    "query": {
        # The function_score allows you to modify the score of documents that are retrieved by a query.
        "function_score": {
            "query": {
                "match": {
                    "name": {
                        "query": "java"
                    }
                }
            },
            "script_score": {
                "script": {
                    "source": """
                    String way1 = doc['name.raw'].value;
                    String way2 = params._source.name;
                    
                    return doc['vote'].value
                    """""
                }
            },
            "boost_mode": "replace"
        }
    }
}
'''
GET es0/_search
`body`
'''
es.search(index='es0', body=body)

{'took': 9,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 100.0,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 100.0,
    '_source': {'name': 'Java集合详解', 'vote': 100}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '1',
    '_score': 30.0,
    '_source': {'name': 'Java多线程详解,Java锁详解', 'vote': 30}}]}}

In [6]:
es.search(index='es0', body=
{
    "query": {
        "function_score": {
            "query": {
                "match": {
                    "name": {
                        "query": "java"
                    }
                }
            },
            "script_score": {
                "script": {
                    "params": {
                        "a": 5,
                        "b": 1.2
                    },
                    "source": "params.a / Math.pow(params.b, doc['vote'].value)"
                }
            }
        }
    }
}
          )

{'took': 8,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 0.0047464985,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '1',
    '_score': 0.0047464985,
    '_source': {'name': 'Java多线程详解,Java锁详解', 'vote': 30}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 1.31609825e-08,
    '_source': {'name': 'Java集合详解', 'vote': 100}}]}}

In [7]:
es.indices.delete('es0')

{'acknowledged': True}